<a href="https://colab.research.google.com/github/ThePlano/Inteligencia_Artificial/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# this class was based on Jere Xu article available at:
# https://towardsdatascience.com/how-to-create-a-chatbot-with-python-deep-learning-in-less-than-an-hour-56a063bdfc44
# thanks to Jere Xu ;)



import numpy as np
import json
import pickle

import nltk
import ssl
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

#nltk.download()

from nltk.stem import WordNetLemmatizer

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD


import random

class ChatBot:
    words = []
    classes = []
    documents = []
    intents = []
    model = []
    ignore_words = ['?', '!']
    lemmatizer = WordNetLemmatizer()

    def createModel(self):

        nltk.download('punkt')
        nltk.download('punkt_tab')
        nltk.download('wordnet')
        nltk.download('omw-1.4')

        data_file = open('intents.json', encoding="utf8").read()
        self.intents = json.loads(data_file)
        for intent in self.intents['intents']:
            for pattern in intent['patterns']:

                # take each word and tokenize it
                w = nltk.word_tokenize(pattern)
                self.words.extend(w)
                # adding documents
                self.documents.append((w, intent['tag']))

                # adding classes to our class list
                if intent['tag'] not in self.classes:
                    self.classes.append(intent['tag'])

        self.words = [self.lemmatizer.lemmatize(w.lower()) for w in self.words if w not in self.ignore_words]
        self.words = sorted(list(set(self.words)))

        self.classes = sorted(list(set(self.classes)))

        print(len(self.documents), "documents")

        print(len(self.classes), "classes", self.classes)

        print(len(self.words), "unique lemmatized words", self.words)

        pickle.dump(self.words, open('words.pkl', 'wb'))
        pickle.dump(self.classes, open('classes.pkl', 'wb'))

        # initializing training data
        training = []
        output_empty = [0] * len(self.classes)
        for doc in self.documents:
            # initializing bag of words
            bag = []
            # list of tokenized words for the pattern
            pattern_words = doc[0]
            # lemmatize each word - create base word, in attempt to represent related words
            pattern_words = [self.lemmatizer.lemmatize(word.lower()) for word in pattern_words]
            # create our bag of words array with 1, if word match found in current pattern
            for w in self.words:
                bag.append(1) if w in pattern_words else bag.append(0)

            # output is a '0' for each tag and '1' for current tag (for each pattern)
            output_row = list(output_empty)
            output_row[self.classes.index(doc[1])] = 1

            training.append([bag, output_row])
        # shuffle our features and turn into np.array
        random.shuffle(training)
        training = np.array(training,dtype=object)
        # create train and test lists. X - patterns, Y - intents
        train_x = list(training[:, 0])
        train_y = list(training[:, 1])
        print("Training data created")

        # Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
        # equal to number of intents to predict output intent with softmax
        model = Sequential()
        model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(64, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(len(train_y[0]), activation='softmax'))

        # Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
        sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
        model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

        # fitting and saving the model
        hist = model.fit(np.array(train_x), np.array(train_y), epochs=2000, batch_size=5, verbose=1)
        model.save('chatbot_model.h5', hist)

        self.model = model

        print("model created")

    def loadModel(self):
        from keras.models import load_model
        self.model = load_model('chatbot_model.h5')
        self.intents = json.loads(open('intents.json').read())
        self.words = pickle.load(open('words.pkl', 'rb'))
        self.classes = pickle.load(open('classes.pkl', 'rb'))

    def clean_up_sentence(self, sentence):
        sentence_words = nltk.word_tokenize(sentence)
        sentence_words = [self.lemmatizer.lemmatize(word.lower()) for word in sentence_words]
        return sentence_words


    def bow(self,sentence, words, show_details=True):
        # tokenize the pattern
        sentence_words = self.clean_up_sentence(sentence)
        # bag of words - matrix of N words, vocabulary matrix
        bag = [0] * len(words)
        for s in sentence_words:
            for i, w in enumerate(words):
                if w == s:
                    # assign 1 if current word is in the vocabulary position
                    bag[i] = 1
                    if show_details:
                        print("found in bag: %s" % w)
        return (np.array(bag))


    def predict_class(self,sentence, model):
        # filter out predictions below a threshold
        p = self.bow(sentence, self.words,show_details=False)
        res = model.predict(np.array([p]))[0]
        ERROR_THRESHOLD = 0.25
        results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
        # sort by strength of probability
        results.sort(key=lambda x: x[1], reverse=True)
        return_list = []
        for r in results:
            return_list.append({"intent": self.classes[r[0]], "probability": str(r[1])})
        ########## print(return_list)
        return return_list

    def getResponse(self, ints, intents_json):
        tag = ints[0]['intent']
        list_of_intents = intents_json['intents']
        for i in list_of_intents:
            if(i['tag']== tag):
                ######### print("achou tag %s"%(tag))
                result = random.choice(i['responses'])
                break
        return result

    def chatbot_response(self, msg):
        ints = self.predict_class(msg, self.model)
        res = self.getResponse(ints, self.intents)
        return res, ints

In [ ]:
Z#pip install nltk numpy tensorflow

from chatbot import ChatBot
myChatBot = ChatBot()


#apenas carregar um modelo pronto
#myChatBot.loadModel()

#criar o modelo
myChatBot.createModel()




print("Bem vindo ao Chatbot")


pergunta = input("como posso te ajudar?")
resposta, intencao = myChatBot.chatbot_response(pergunta)
print(resposta + "   ["+intencao[0]['intent']+"]")


while (intencao[0]['intent']!="despedida"):
    pergunta = input("posso lhe ajudar com algo a mais?")
    resposta, intencao = myChatBot.chatbot_response(pergunta)
    print(resposta + "   [" + intencao[0]['intent'] + "]")

print("Foi um prazer atendê-lo")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


24 documents
6 classes ['area_prioritaria', 'data_prova', 'despedida', 'emprestimo', 'proposta', 'saudacao']
40 unique lemmatized words ['a', 'ao', 'apoio', 'beleza', 'bem', 'como', 'da', 'dia', 'em', 'empréstimo', 'exame', 'fapesp', 'fui', 'há', 'no', 'o', 'obrigado', 'oi', 'ola', 'opa', 'pipe', 'posso', 'prioritárias', 'programa', 'proposta', 'prova', 'quando', 'que', 'serum', 'será', 'submeter', 'tchau', 'ter', 'tudo', 'um', 'uma', 'vai', 'valeu', 'áreas', 'é']
Training data created
Epoch 1/2000
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.1639 - loss: 1.7817
Epoch 2/2000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.3139 - loss: 1.7394
Epoch 3/2000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4444 - loss: 1.6990
Epoch 4/2000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3639 - loss: 1.6438 
Epoch 5/2000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4139 - loss: 1.5897 
Epoch 6/2000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4667 - loss: 1.4505 


model created
Bem vindo ao Chatbot
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
As propostas são recebidas o ano todo   [proposta]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Não. O PIPE visa apoiar pesquisas promissoras, com potencial de inovação, em qualquer área do conhecimento   [area_prioritaria]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
O apoio da FAPESP não é realizado na forma de empréstimo   [emprestimo]
posso lhe ajudar com algo a mais?Obrigado
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Até breve   [despedida]
Foi um prazer atendê-lo
